<a href="https://colab.research.google.com/github/knoel99/learn_cuda/blob/master/01_easier_intro_to_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An even easier introduction to CUDA

Source: https://developer.nvidia.com/blog/even-easier-introduction-cuda/

Many noob notes for C++ are added.

# Requirements
- Learn how to run C++ code in colab
- Select a colab runtime with a GPU

In [43]:
# Test C++ code
%%writefile hello.cpp
#include <iostream>
using namespace std;

int main() {
  cout << "Hello from Colab!" << endl;
  return 0;
}

Overwriting hello.cpp


In [44]:
# Compile with g++
!g++ hello.cpp -o hello
!./hello

Hello from Colab!


# Noob notes:
- `<iostream>` is the library needed to print results in the terminal
- writing `using namespace std;` allows to directly write function `cout` instead of `std::cout`
- `cout`means "console output" or "character output"

# Addition of two arrays with standard C++ code, on CPU

In this tutorial the studied function is just the addition of two arrays with 1 million elements each.

In [46]:
%%writefile addition.cpp
#include <iostream>
#include <math.h>

// Add two arrays
void add(int n, float *x, float *y) {
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void) {
  int N = 1<<20; // 1 M elements

  float *x = new float[N];
  float *y = new float[N];

  // Init the two arrays with a for loop.
  // tutorial says : init arrays on the host => host is the CPU
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all elements should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N ; i++)
    maxError = fmax(maxError, fabs(y[i] - 3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete[] x;
  delete[] y;

  return 0;

}

Overwriting addition.cpp


In [47]:
# Compile and run
!g++ addition.cpp -o addition
!./addition

Max error: 0


No error in the addition as expected

# Noob note

Meaning of `int N = 1<<20; `

- `1<<20` means 2^20, where the double chevron means shifting bits to the left. The two arrays has 1 048 676 elements.
- Each element of the array is a float, defined on 4 bytes.
- Each array is then about 4*2^20 bytes=~ 4 MB in memory


Some examples:
- 1 << 10 = 1024 ~ 1 kB
- 1 << 20 = 1 048 576 ~1 MB
- 1 << 30 = 1 073 741 824  ~1 GB

Why put the pointers in the function arguments instead of the arrays themselves, just like in python ?


In python we have:
```python
def add(a, b):
    for i in range(len(a)):
        b[i] = a[i] + b[i]

x = [1.0] * 2**20
y = [2.0] * 2**20
add(x, y)
```

In Cpp:
```cpp
void add(int n, float *x, float *y) {
    for (int i = 0; i < n; i++)
        y[i] = x[i] + y[i];
}
int N = 1<<20;
float *x = new float[N];
float *y = new float[N];

for (int i = 0; i < N; i++) {
  x[i] = 1.0f;
  y[i] = 2.0f;
}

add(N, x, y);
```

In theory those two lines are equivalent, but the convention is to declare the pointer of the variable instead of the variable itself.



# Running the addition on the GPU

Now I want to run the addtion function onto the GPU, using its cores.

We have to turn this C++ function into a kernel, ie a function that can run on the GPU.

To do this, we just need to add the keyword `__global__` to the function. The CUDA C++ compiler can then run the function on the GPU.

Defitions:
- CUDA kernel: a function that can be run on the GPU
- Device code: the code that runs on the GPU
- Host code: the code that runs on the CPU

Example of the tutorial:

```cpp
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *sum, float *x, float *y) {
  for (int i = 0; i < n; i++)
  sum[i] = x[i] + y[i];
}
```

# Memory allocation in CUDA

In standard C++, to allocate the memory for two arrays we do:

```cpp
// Init
int N = 10;
float *x = new float[N]:
float *y = new float[N];

// Do some stuff

// Free memory
delete[] x
delete[] y
```

In CUDA, thanks to the Unified Memory concept, the equivalent can be written as:

```cpp
// Allocate Unified Memory --- accessible from CPU or GPU
float *x, *y;
cudaMallocManaged(&x, N * sizeof(float));
cudaMallocManaged(&y, N * sizeof(float));

// Do some stuff

// Free memory
cudaFree(x);
cudaFree(y)
```

So now we have the kernel defined with the keyword `__global__` like this:

```cpp
__global__
void add(int n, float *x, float *y){
  ...
}
```

And to call the kernel from the host in the `main` function, we do this:

```cpp
int main() {
  // Stuff before

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, sum, x, y);

  // Wait for the GPU to finish before accessing on host
  cudaDeviceSynschronize();

  // Stuff after
}
```

Where `cudaDeviceSynchronize()` is needed make the CPU wait untill the computation on the GPU is finished.

The complete code with the kernel is then:

In [48]:
%%writefile add_cuda.cu
#include <iostream>
#include <math.h>

// Kernel function to add two arrays:
__global__
void add(int n, float *x, float *y){
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void) {
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory - accessible from CPU or GPU
  cudaMallocManaged(&x, N * sizeof(float));
  cudaMallocManaged(&y, N * sizeof(float));

  // Init the two arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    std::cout << "CUDA error: " << cudaGetErrorString(err) << std::endl;
  }

  // Wait for the GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++) {
    maxError = fmax(maxError, fabs(y[i] - 3.0f));
  }
  std::cout << "Max error: " << maxError << std:: endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}


Overwriting add_cuda.cu


The file below is to fix the compatibility issue between CUDA and the GPU. More details below.

In [49]:
%%writefile detect_cuda_compiler.cu
// This file is used to adapt the version
#include <cuda_runtime.h>
#include <iostream>

int main() {
  int deviceCount = 0;
  cudaGetDeviceCount(&deviceCount);
  if (deviceCount == 0) {
    std::cout << "No CUDA-capable device found." << std::endl;
    return 1;
  }

  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, 0);
  std::cout << "sm_" << prop.major << prop.minor << std::endl;
  // Output is sm_75 for T4 on Colab
  return 0;
}

Writing detect_cuda_compiler.cu


In [53]:
# Run this code to get the version for the option -arch=sm_xx
%%bash
nvcc detect_cuda_compiler.cu -o detect_cuda_compiler
./detect_cuda_compiler

ARCH=$(./detect_cuda_compiler)
echo $ARCH

nvcc -arch=$ARCH add_cuda.cu -o add_cuda
./add_cuda

sm_75
sm_75
Max error: 0


Max error is 0, everything works well. We run our first kernel :)

## Error due to version mismatch between the compiler nvcc and the driver of the GPU.

You may get an error where the maxError is not 0 after the kernel is run. You should add this snippet to catch the CUDA error:

```cpp
// Run kernel on 1M elements on the GPU
add<<<1, 1>>>(N, x, y);

cudaError_t err = cudaGetLastError();
if (err != cudaSuccess) {
  std::cout << "CUDA error: " << cudaGetErrorString(err) << std::endl;
}
```

For me the error was:
```sh
CUDA error: the provided PTX was compiled with an unsupported toolchain.
Max error: 1
```

The code in `detect_cuda_compiler.cu` fixes the issue.

### Understanding the error

To understand the error is here the workflow of how a kernel is compiled and exectued on a GPU.

1. The source code is written in a `.cu`file.

2. The source code is compiled into PTX (Parallel Thread Exectuion), which is the equivalent of bycode in Java. The command to compile is `nvcc add_cuda.cu -o add_cuda`. CUDA source code is not directly compiled into binary code because we need to keep compatibility between the different hardware architecture (Turing, Ampere, Ada, Blackwell, etc...).

3. The kernel is run with `./add_cuda`.


In this process, multiple elements must align:

- CUDA Toolkit version: this is the version that appears when running `nvcc --version`.
- Driver CUDA Capability version: this is also called Max Supported CUDA version. This is the version that appears when running `nvidia-smi` (smi means system management interface).
- Compute Capability: this is the version of the architecture of the hardware. This is the version that appears when running `nvidia-smi --query-gpu=compute_cap --format=csv`




```bash
Options for steering GPU code generation.
=========================================

--gpu-architecture <arch>                       (-arch)                         
        Specify the name of the class of NVIDIA 'virtual' GPU architecture for which
        the CUDA input files must be compiled.
        With the exception as described for the shorthand below, the architecture
        specified with this option must be a 'virtual' architecture (such as compute_50).
        Normally, this option alone does not trigger assembly of the generated PTX
        for a 'real' architecture (that is the role of nvcc option '--gpu-code',
        see below); rather, its purpose is to control preprocessing and compilation
        of the input to PTX.
        For convenience, in case of simple nvcc compilations, the following shorthand
        is supported.  If no value for option '--gpu-code' is specified, then the
        value of this option defaults to the value of '--gpu-architecture'.  In this
        situation, as only exception to the description above, the value specified
        for '--gpu-architecture' may be a 'real' architecture (such as a sm_50),
        in which case nvcc uses the specified 'real' architecture and its closest
        'virtual' architecture as effective architecture values.  For example, 'nvcc
        --gpu-architecture=sm_50' is equivalent to 'nvcc --gpu-architecture=compute_50
        --gpu-code=sm_50,compute_50'.
        -arch=all         build for all supported architectures (sm_*), and add PTX
        for the highest major architecture to the generated code.
        -arch=all-major   build for just supported major versions (sm_*0), plus the
        earliest supported, and add PTX for the highest major architecture to the
        generated code.
        -arch=native      build for all architectures (sm_*) on the current system
        Note: -arch=native, -arch=all, -arch=all-major cannot be used with the -code
        option, but can be used with -gencode options.
        Allowed values for this option:  'all','all-major','compute_50','compute_52',
        'compute_53','compute_60','compute_61','compute_62','compute_70','compute_72',
        'compute_75','compute_80','compute_86','compute_87','compute_89','compute_90',
        'compute_90a','lto_50','lto_52','lto_53','lto_60','lto_61','lto_62','lto_70',
        'lto_72','lto_75','lto_80','lto_86','lto_87','lto_89','lto_90','lto_90a',
        'native','sm_50','sm_52','sm_53','sm_60','sm_61','sm_62','sm_70','sm_72',
        'sm_75','sm_80','sm_86','sm_87','sm_89','sm_90','sm_90a'.

--gpu-code <code>,...                           (-code)                         
        Specify the name of the NVIDIA GPU to assemble and optimize PTX for.
        nvcc embeds a compiled code image in the resulting executable for each specified
        <code> architecture, which is a true binary load image for each 'real' architecture
        (such as sm_50), and PTX code for the 'virtual' architecture (such as compute_50).
        During runtime, such embedded PTX code is dynamically compiled by the CUDA
        runtime system if no binary load image is found for the 'current' GPU.
        Architectures specified for options '--gpu-architecture' and '--gpu-code'
        may be 'virtual' as well as 'real', but the <code> architectures must be
        compatible with the <arch> architecture.  When the '--gpu-code' option is
        used, the value for the '--gpu-architecture' option must be a 'virtual' PTX
        architecture.
        For instance, '--gpu-architecture=compute_60' is not compatible with '--gpu-code=sm_52',
        because the earlier compilation stages will assume the availability of 'compute_60'
        features that are not present on 'sm_52'.
        Allowed values for this option:  'compute_50','compute_52','compute_53',
        'compute_60','compute_61','compute_62','compute_70','compute_72','compute_75',
        'compute_80','compute_86','compute_87','compute_89','compute_90','compute_90a',
        'lto_50','lto_52','lto_53','lto_60','lto_61','lto_62','lto_70','lto_72',
        'lto_75','lto_80','lto_86','lto_87','lto_89','lto_90','lto_90a','sm_50',
        'sm_52','sm_53','sm_60','sm_61','sm_62','sm_70','sm_72','sm_75','sm_80',
        'sm_86','sm_87','sm_89','sm_90','sm_90a'.

--list-gpu-code                                 (-code-ls)                      
        List the non-accelerated gpu architectures (sm_XX) supported by the compiler
        and exit. If both --list-gpu-code and --list-gpu-arch are set, the list is
        displayed using the same format as the --generate-code value.

--list-gpu-arch                                 (-arch-ls)                      
        List the non-accelerated virtual device architectures (compute_XX) supported
        by the compiler and exit. If both --list-gpu-code and --list-gpu-arch are
        set, the list is displayed using the same format as the --generate-code value.

```

# Noob Note

Let's check out ourselves that Unified Memory is accessile from the CPU or the GPU

In [78]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [77]:
!nvcc --list-gpu-code

sm_50
sm_52
sm_53
sm_60
sm_61
sm_62
sm_70
sm_72
sm_75
sm_80
sm_86
sm_87
sm_89
sm_90


In [82]:
!nvcc --list-gpu-arch

compute_50
compute_52
compute_53
compute_60
compute_61
compute_62
compute_70
compute_72
compute_75
compute_80
compute_86
compute_87
compute_89
compute_90


In [86]:
!nvcc -h


Usage  : nvcc [options] <inputfile>

Options for specifying the compilation phase
More exactly, this option specifies up to which stage the input files must be compiled,
according to the following compilation trajectories for different input file types:
        .c/.cc/.cpp/.cxx : preprocess, compile, link
        .o               : link
        .i/.ii           : compile, link
        .cu              : preprocess, cuda frontend, PTX assemble,
                           merge with host C code, compile, link
        .gpu             : cicc compile into cubin
        .ptx             : PTX assemble into cubin.

--cuda                                          (-cuda)                         
        Compile all .cu input files to .cu.cpp.ii output.

--cubin                                         (-cubin)                        
        Compile all .cu/.gpu/.ptx input files to device-only .cubin files.  This
        step discards the host code for each .cu input file.

--fatbin          

In [74]:
!nvidia-smi

Sun Nov  9 17:35:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [75]:
!nvidia-smi -h

NVIDIA System Management Interface -- v550.54.15

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- L